In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime as dt
import math
from scipy.stats import norm
import datetime
from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum
from typing import Union

In [2]:
class TimeProcessor:
    def __init__(self, expiration_date):
        self.expiration_date = self.parse_expiration_date(expiration_date)

    def parse_expiration_date(self, expiration_date):
        if isinstance(expiration_date, datetime.datetime):
            return expiration_date
        elif isinstance(expiration_date, str):
            for date_format in ['%Y-%m-%d','%d-%m-%Y','%m-%d-%Y', '%m/%d/%Y', '%d/%m/%Y',"%Y-%m-%d"]:
                try:
                    return datetime.datetime.strptime(expiration_date, date_format)
                except ValueError:
                    pass
            raise ValueError("Unsupported date format.")
        else:
            raise TypeError("expiration_date must be a string or a datetime.datetime object.")

    def calculate_time_to_expiration(self):
        today = datetime.datetime.now()
        if self.expiration_date < today:
            return 0  # Return 0 if the expiration date is in the past
        time_to_expiration = (self.expiration_date - today)
        return time_to_expiration

In [3]:
class Option:

    def __init__(self, asset_class, option_type, strike_price, expiration_date, number_of_contracts, underlying_price, risk_free_rate, volatility):

        self.asset_class = asset_class

        self.option_type = option_type

        self.strike_price = strike_price

        self.expiration_date = self.parse_expiration_date(expiration_date)

        self.number_of_contracts = number_of_contracts

        self.underlying_price = underlying_price

        self.risk_free_rate = risk_free_rate

        self.volatility = volatility

    def parse_expiration_date(self, expiration_date):

        if isinstance(expiration_date, datetime.datetime):

            return expiration_date

        elif isinstance(expiration_date, str):

            for date_format in ['%Y-%m-%d', '%m/%d/%Y', '%d-%m-%Y', '%d/%m/%Y']:

                try:

                    return datetime.datetime.strptime(expiration_date, date_format)

                except ValueError:

                    pass

            raise ValueError("Unsupported date format. Please use 'YYYY-MM-DD', 'MM/DD/YYYY', 'DD-MM-YYYY', or 'DD/MM/YYYY'.")

        else:

            raise TypeError("expiration_date must be a string or a datetime.datetime object.")

    def calculate_option_premium(self):

        self.T = TimeProcessor(self.expiration_date).calculate_time_to_expiration().days / 365

        d1 = (math.log(self.underlying_price / self.strike_price) + (self.risk_free_rate + self.volatility**2 / 2) * self.T) / (self.volatility * math.sqrt(self.T))

        d2 = d1 - self.volatility * math.sqrt(self.T)

        if self.asset_class == "Call" and self.option_type == "Long":

            return self.number_of_contracts * (self.underlying_price * norm.cdf(d1) - self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(d2))

        elif self.asset_class == "Call" and self.option_type == "Short":

            return -self.number_of_contracts * (self.underlying_price * norm.cdf(d1) - self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(d2))

        elif self.asset_class == "Put" and self.option_type == "Long":

            return self.number_of_contracts * (self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(-d2) - self.underlying_price * norm.cdf(-d1))

        elif self.asset_class == "Put" and self.option_type == "Short":

            return -self.number_of_contracts * (self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(-d2) - self.underlying_price * norm.cdf(-d1))

        else:

            raise ValueError("Only long/short call and put options are supported.")

    def calculate_option_theta(self):

        self.T = TimeProcessor(self.expiration_date).calculate_time_to_expiration().days / 365

        d1 = (math.log(self.underlying_price / self.strike_price) + (self.risk_free_rate + self.volatility**2 / 2) * self.T) / (self.volatility * math.sqrt(self.T))

        d2 = d1 - self.volatility * math.sqrt(self.T)

        if self.asset_class == "Call" and self.option_type == "Long":

            return -(self.number_of_contracts * (self.underlying_price * norm.pdf(d1) * self.volatility / (2 * math.sqrt(self.T)) + self.risk_free_rate * self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(d2)))

        elif self.asset_class == "Call" and self.option_type == "Short":

            return self.number_of_contracts * (self.underlying_price * norm.pdf(d1) * self.volatility / (2 * math.sqrt(self.T)) + self.risk_free_rate * self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(d2))

        elif self.asset_class == "Put" and self.option_type == "Long":

            return -(self.number_of_contracts * (self.underlying_price * norm.pdf(d1) * self.volatility / (2 * math.sqrt(self.T)) - self.risk_free_rate * self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(-d2)))

        elif self.asset_class == "Put" and self.option_type == "Short":

            return self.number_of_contracts * (self.underlying_price * norm.pdf(d1) * self.volatility / (2 * math.sqrt(self.T)) - self.risk_free_rate * self.strike_price * math.exp(-self.risk_free_rate * self.T) * norm.cdf(-d2))

        else:

            raise ValueError("Only long/short call and put options are supported.")

    def calculate_option_vega(self):

        self.T = TimeProcessor(self.expiration_date).calculate_time_to_expiration().days / 365

        d1 = (math.log(self.underlying_price / self.strike_price) + (self.risk_free_rate + self.volatility**2 / 2) * self.T) / (self.volatility * math.sqrt(self.T))

        return self.number_of_contracts * self.underlying_price * math.sqrt(self.T) * norm.pdf(d1)

    def calculate_option_gamma(self):

        self.T = TimeProcessor(self.expiration_date).calculate_time_to_expiration().days / 365

        d1 = (math.log(self.underlying_price / self.strike_price) + (self.risk_free_rate + self.volatility**2 / 2) * self.T) / (self.volatility * math.sqrt(self.T))

        return norm.pdf(d1) / (self.number_of_contracts * self.underlying_price * self.volatility * math.sqrt(self.T))

    def calculate_option_delta(self):

        self.T = TimeProcessor(self.expiration_date).calculate_time_to_expiration().days / 365

        d1 = (math.log(self.underlying_price / self.strike_price) + (self.risk_free_rate + self.volatility**2 / 2) * self.T) / (self.volatility * math.sqrt(self.T))

        if self.asset_class == "Call" and self.option_type == "Long":

            return norm.cdf(d1)

        elif self.asset_class == "Call" and self.option_type == "Short":

            return -norm.cdf(d1)

        elif self.asset_class == "Put" and self.option_type == "Long":

            return norm.cdf(d1) - 1

        elif self.asset_class == "Put" and self.option_type == "Short":

            return 1 - norm.cdf(d1)

        else:

            raise ValueError("Only long/short call and put options are supported.")

    def calculate_option_rho(self):

        self.T = TimeProcessor(self.expiration_date).calculate_time_to_expiration().days / 365

        d1 = (math.log(self.underlying_price / self.strike_price) + (self.risk_free_rate + self.volatility**2 / 2) * self.T) / (self.volatility * math.sqrt(self.T))

        d2 = d1 - self.volatility * math.sqrt(self.T)

        if self.asset_class == "Call" and self.option_type == "Long":

            return self.number_of_contracts * self.strike_price * self.T * math.exp(-self.risk_free_rate * self.T) * norm.cdf(d2)

        elif self.asset_class == "Call" and self.option_type == "Short":

            return -self.number_of_contracts * self.strike_price * self.T * math.exp(-self.risk_free_rate * self.T) * norm.cdf(d2)

        elif self.asset_class == "Put" and self.option_type == "Long":

            return -self.number_of_contracts * self.strike_price * self.T * math.exp(-self.risk_free_rate * self.T) * norm.cdf(-d2)

        elif self.asset_class == "Put" and self.option_type == "Short":

            return self.number_of_contracts * self.strike_price * self.T * math.exp(-self.risk_free_rate * self.T) * norm.cdf(-d2)

        else:

            raise ValueError("Only long/short call and put options are supported.")

    def monitor_position(self):

        delta = self.calculate_option_delta()

        gamma = self.calculate_option_gamma()

        underlying_price = self.underlying_price

        number_of_contracts = self.number_of_contracts

        strike_price = self.strike_price

        risk_free_rate = self.risk_free_rate

        volatility = self.volatility

        T = self.T

        # Calculate the required number of contracts for the underlying asset

        underlying_contracts = -delta * number_of_contracts

        # Calculate the required number of contracts for the option

        option_contracts = gamma * number_of_contracts * underlying_price * math.sqrt(T)

        # Print the advice

        print(f"Delta is {delta}, needs adjustment. Hedge with {underlying_contracts} contracts of the underlying asset.")

        print(f"Gamma is {gamma}, needs adjustment. Hedge with {option_contracts} contracts of the option.")

class Strategy:

    def __init__(self):

        self.positions = []

    def add(self, position):

        self.positions.append(position)

    def monitor_portfolio(self):

        for position in self.positions:

            position.monitor_position()

    def print_portfolio_greeks(self):

        total_delta = 0

        total_gamma = 0

        total_theta = 0

        total_vega = 0

        total_rho = 0

        for position in self.positions:

            total_delta += position.calculate_option_delta()

            total_gamma += position.calculate_option_gamma()

            total_theta += position.calculate_option_theta()

            total_vega += position.calculate_option_vega()

            total_rho += position.calculate_option_rho()

        print(f"Total Delta: {total_delta}")

        print(f"Total Gamma: {total_gamma}")

        print(f"Total Theta: {total_theta}")

        print(f"Total Vega: {total_vega}")

        print(f"Total Rho: {total_rho}")

In [9]:
# import tkinter as tk
# from tkinter import messagebox
# from tkinter import ttk

# class PortfolioGUI:
#     def __init__(self, master):
#         self.master = master
#         master.title("Option Portfolio Simulator")

#         # Create a frame for the positions
#         self.positions_frame = tk.Frame(master)
#         self.positions_frame.pack(padx=10, pady=10)

#         # Create a label and entry fields for the position attributes
#         tk.Label(self.positions_frame, text="Asset Class:").grid(row=0, column=0)
#         self.asset_class = tk.StringVar()
#         tk.OptionMenu(self.positions_frame, self.asset_class, "Call", "Put").grid(row=0, column=1)

#         tk.Label(self.positions_frame, text="Option Type:").grid(row=1, column=0)
#         self.option_type = tk.StringVar()
#         tk.OptionMenu(self.positions_frame, self.option_type, "Long", "Short").grid(row=1, column=1)

#         tk.Label(self.positions_frame, text="Strike Price:").grid(row=2, column=0)
#         self.strike_price = tk.StringVar()
#         tk.Entry(self.positions_frame, textvariable=self.strike_price).grid(row=2, column=1)

#         tk.Label(self.positions_frame, text="Expiration Date:").grid(row=3, column=0)
#         self.expiration_date = tk.StringVar()
#         tk.Entry(self.positions_frame, textvariable=self.expiration_date).grid(row=3, column=1)

#         tk.Label(self.positions_frame, text="Number of Contracts:").grid(row=4, column=0)
#         self.number_of_contracts = tk.StringVar()
#         tk.Entry(self.positions_frame, textvariable=self.number_of_contracts).grid(row=4, column=1)

#         tk.Label(self.positions_frame, text="Underlying Price:").grid(row=5, column=0)
#         self.underlying_price = tk.StringVar()
#         tk.Entry(self.positions_frame, textvariable=self.underlying_price).grid(row=5, column=1)

#         tk.Label(self.positions_frame, text="Risk-Free Rate:").grid(row=6, column=0)
#         self.risk_free_rate = tk.StringVar()
#         tk.Entry(self.positions_frame, textvariable=self.risk_free_rate).grid(row=6, column=1)

#         tk.Label(self.positions_frame, text="Volatility:").grid(row=7, column=0)
#         self.volatility = tk.StringVar()
#         tk.Entry(self.positions_frame, textvariable=self.volatility).grid(row=7, column=1)

#         # Create a button to add a position
#         self.add_position_button = tk.Button(self.positions_frame, text="Add Position", command=self.add_position)
#         self.add_position_button.grid(row=8, column=0, columnspan=2)

#         # Create a button to remove a position
#         self.remove_position_button = tk.Button(self.positions_frame, text="Remove Position", command=self.remove_position)
#         self.remove_position_button.grid(row=9, column=0, columnspan=2)

#        # Create a button to go back to the main menu
#         self.back_to_menu_button = tk.Button(self.positions_frame, text="Back to Menu", command=self.go_to_menu)
#         self.back_to_menu_button.grid(row=10, column=0, columnspan=2)

#         # Create a label to display the portfolio Greeks
#         self.greeks_label = tk.Label(master, text="")
#         self.greeks_label.pack(padx=10, pady=10)

#         # Create a button to get an advice
#         self.advice_button = tk.Button(master, text="Get Advice", command=self.get_advice)
#         self.advice_button.pack(padx=10, pady=10)

#         # Create an instance of the Strategy class
#         self.strategy = Strategy()

#     def add_position(self):
#         # Retrieve the position attributes from the input fields
#         asset_class = self.asset_class.get()
#         option_type = self.option_type.get()
#         strike_price = float(self.strike_price.get())
#         expiration_date = self.expiration_date.get()
#         number_of_contracts = int(self.number_of_contracts.get())
#         underlying_price = float(self.underlying_price.get())
#         risk_free_rate = float(self.risk_free_rate.get())
#         volatility = float(self.volatility.get())

#         # Add the position to the portfolio
#         self.strategy.add(Option(asset_class, option_type, strike_price, expiration_date, number_of_contracts, underlying_price, risk_free_rate, volatility))

#         # Update the GUI to reflect the new position
#         if asset_class and option_type and strike_price and expiration_date and number_of_contracts and underlying_price and risk_free_rate and volatility:
#             self.update_gui()

#     def remove_position(self):
#         # Retrieve the position to remove from the GUI
#         position_index = int(messagebox.askquestion("Remove Position", "Which position do you want to remove?"))

#         # Remove the position from the portfolio
#         del self.strategy.positions[position_index]

#         # Update the GUI to reflect the removed position
#         self.update_gui()

#     def get_advice(self):
#         # Calculate the portfolio Greeks
#         delta = 0
#         gamma = 0
#         theta = 0
#         vega = 0
#         rho = 0

#         for position in self.strategy.positions:
#             delta += position.calculate_option_delta()
#             gamma += position.calculate_option_gamma()
#             theta += position.calculate_option_theta()
#             vega += position.calculate_option_vega()
#             rho += position.calculate_option_rho()

#         # Display the portfolio Greeks
#         self.greeks_label.config(text=f"Delta: {delta}, Gamma: {gamma}, Theta: {theta}, Vega: {vega}, Rho: {rho}")

#     def update_gui(self):
#         # Clear the positions frame
#         for widget in self.positions_frame.winfo_children():
#             widget.destroy()

#         # Re-create the positions frame with the updated positions
#         for position in self.strategy.positions:
#             tk.Label(self.positions_frame, text=f"Asset Class: {position.asset_class}").grid(row=0, column=0)
#             tk.Label(self.positions_frame, text=f"Option Type: {position.option_type}").grid(row=1, column=0)
#             tk.Label(self.positions_frame, text=f"Strike Price: {position.strike_price}").grid(row=2, column=0)
#             tk.Label(self.positions_frame, text=f"Expiration Date: {position.expiration_date}").grid(row=3, column=0)
#             tk.Label(self.positions_frame, text=f"Number of Contracts: {position.number_of_contracts}").grid(row=4, column=0)
#             tk.Label(self.positions_frame, text=f"Underlying Price: {position.underlying_price}").grid(row=5, column=0)
#             tk.Label(self.positions_frame, text=f"Risk-Free Rate: {position.risk_free_rate}").grid(row=6, column=0)
#             tk.Label(self.positions_frame, text=f"Volatility: {position.volatility}").grid(row=7, column=0)
#     def go_to_menu(self):
#         # Clear the positions frame
#         for widget in self.positions_frame.winfo_children():
#             widget.destroy()

# root = tk.Tk()
# app = PortfolioGUI(root)
# root.mainloop()

In [10]:
# import tkinter as tk
# from tkinter import ttk
# from tkinter import messagebox

# class Application(tk.Frame):
#     def __init__(self, master=None):
#         super().__init__(master)
#         self.master = master
#         self.pack()
#         self.create_widgets()
#         self.current_page = "main"

#     def create_widgets(self):
#         self.new_position_button = tk.Button(self)
#         self.new_position_button["text"] = "New Position"
#         self.new_position_button["command"] = self.new_position_window
#         self.new_position_button.pack(side="top")

#         self.position_list_button = tk.Button(self)
#         self.position_list_button["text"] = "Position List"
#         self.position_list_button["command"] = self.position_list_window
#         self.position_list_button.pack(side="top")

#         self.statistics_button = tk.Button(self)
#         self.statistics_button["text"] = "Statistics"
#         self.statistics_button["command"] = self.statistics_window
#         self.statistics_button.pack(side="top")

#         self.sensitivity_analysis_button = tk.Button(self)
#         self.sensitivity_analysis_button["text"] = "Sensitivity Analysis"
#         self.sensitivity_analysis_button["command"] = self.sensitivity_analysis_window
#         self.sensitivity_analysis_button.pack(side="top")

#         self.portfolio_value_button = tk.Button(self)
#         self.portfolio_value_button["text"] = "Portfolio Value"
#         self.portfolio_value_button["command"] = self.portfolio_value_window
#         self.portfolio_value_button.pack(side="top")

#     def new_position_window(self):
#         self.new_position_window = tk.Toplevel(self.master)
#         self.new_position_window.title("New Position")
#         self.new_position_window_frame = tk.Frame(self.new_position_window)
#         self.new_position_window_frame.pack(padx=10, pady=10)
#         self.new_position_label = tk.Label(self.new_position_window_frame, text="Enter new position details:")
#         self.new_position_label.pack(side=tk.LEFT)
#         self.new_position_entry = tk.Entry(self.new_position_window_frame)
#         self.new_position_entry.pack(side=tk.LEFT)

#     def position_list_window(self):
#         self.position_list_window = tk.Toplevel(self.master)
#         self.position_list_window.title("Position List")
#         self.position_list_window_frame = tk.Frame(self.position_list_window)
#         self.position_list_window_frame.pack(padx=10, pady=10)
#         self.position_list_label = tk.Label(self.position_list_window_frame, text="Position List:")
#         self.position_list_label.pack(side=tk.LEFT)
#         self.position_list_entry = tk.Entry(self.position_list_window_frame)
#         self.position_list_entry.pack(side=tk.LEFT)

#     def statistics_window(self):
#         self.statistics_window = tk.Toplevel(self.master)
#         self.statistics_window.title("Statistics")
#         self.statistics_window_frame = tk.Frame(self.statistics_window)
#         self.statistics_window_frame.pack(padx=10, pady=10)
#         self.statistics_label = tk.Label(self.statistics_window_frame, text="Statistics:")
#         self.statistics_label.pack(side=tk.LEFT)
#         self.statistics_entry = tk.Entry(self.statistics_window_frame)
#         self.statistics_entry.pack(side=tk.LEFT)

#     def sensitivity_analysis_window(self):
#         self.sensitivity_analysis_window = tk.Toplevel(self.master)
#         self.sensitivity_analysis_window.title("Sensitivity Analysis")
#         self.sensitivity_analysis_window_frame = tk.Frame(self.sensitivity_analysis_window)
#         self.sensitivity_analysis_window_frame.pack(padx=10, pady=10)
#         self.sensitivity_analysis_label = tk.Label(self.sensitivity_analysis_window_frame, text="Sensitivity Analysis:")
#         self.sensitivity_analysis_label.pack(side=tk.LEFT)
#         self.sensitivity_analysis_entry = tk.Entry(self.sensitivity_analysis_window_frame)
#         self.sensitivity_analysis_entry.pack(side=tk.LEFT)

#     def portfolio_value_window(self):
#         self.portfolio_value_window = tk.Toplevel(self.master)
#         self.portfolio_value_window.title("Portfolio Value")
#         self.portfolio_value_window_frame = tk.Frame(self.portfolio_value_window)
#         self.portfolio_value_window_frame.pack(padx=10, pady=10)
#         self.portfolio_value_label = tk.Label(self.portfolio_value_window_frame, text="Enter portfolio value:")
#         self.portfolio_value_label.pack(side=tk.LEFT)
#         self.portfolio_value_entry = tk.Entry(self.portfolio_value_window_frame)
#         self.portfolio_value_entry.pack(side=tk.LEFT)

#         self.portfolio_value_button = tk.Button(self.portfolio_value_window_frame)
#         self.portfolio_value_button["text"] = "Display Portfolio Value"
#         self.portfolio_value_button["command"] = self.display_portfolio_value
#         self.portfolio_value_button.pack(side=tk.LEFT)

#     def display_portfolio_value(self):
#         portfolio_value = self.portfolio_value_entry.get()
#         self.portfolio_value_label = tk.Label(self.master, text="Portfolio Value: " + portfolio_value)
#         self.portfolio_value_label.pack(side=tk.LEFT)

#     def switch_page(self, page):
#         if self.current_page == "main":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif self.current_page == "new_position":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif self.current_page == "position_list":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif self.current_page == "statistics":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif self.current_page == "sensitivity_analysis":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif self.current_page == "portfolio_value":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()

#         if page == "main":
#             self.new_position_button.pack(side="top")
#             self.position_list_button.pack(side="top")
#             self.statistics_button.pack(side="top")
#             self.sensitivity_analysis_button.pack(side="top")
#             self.portfolio_value_button.pack(side="top")
#         elif page == "new_position":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif page == "position_list":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif page == "statistics":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif page == "sensitivity_analysis":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()
#         elif page == "portfolio_value":
#             self.new_position_button.pack_forget()
#             self.position_list_button.pack_forget()
#             self.statistics_button.pack_forget()
#             self.sensitivity_analysis_button.pack_forget()
#             self.portfolio_value_button.pack_forget()

#         self.current_page = page

# root = tk.Tk()
# app = Application(master=root)
# app.mainloop()

In [12]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/add_position', methods=['POST'])
def add_position():
    data = request.get_json()
    # Process the position data and add it to the portfolio
    # Calculate the portfolio Greeks
    delta = 0
    gamma = 0
    theta = 0
    vega = 0
    rho = 0
    # ...
    return jsonify({'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1